In [1]:
!cp -r /kaggle/input/doubleroc/* /kaggle/working/SASV_DoubleRoc-main

In [2]:
!ln -s /kaggle/input/nndang-fusion-test /kaggle/working/SASV_DoubleRoc-main/fusion_test

In [3]:
!ln -s /kaggle/input/nndang-spoofing-data /kaggle/working/SASV_DoubleRoc-main/spoofing-data

In [4]:
!ln -s /kaggle/input/nndang-vn-celeb/VLSP23_VN_celeb2 /kaggle/working/SASV_DoubleRoc-main/vn-celeb

In [5]:
!ln -s /kaggle/input/weight-cm /kaggle/working/SASV_DoubleRoc-main/CM_weight

In [6]:
!ln -s /kaggle/input/tst-enr /kaggle/working/SASV_DoubleRoc-main/enrollment_txt

In [7]:
!ln -s /kaggle/input/ecapa-tdnn /kaggle/working/SASV_DoubleRoc-main/ecapa-tdnn

In [8]:
!ln -s /kaggle/input/vn-celeb-txt-final /kaggle/working/SASV_DoubleRoc-main/vn_celeb_txt

In [9]:
!ln -s /kaggle/input/emb-roc /kaggle/working/SASV_DoubleRoc-main/emb_roc

In [10]:
!ln -s /kaggle/input/public-tst /kaggle/working/SASV_DoubleRoc-main/public_tst

In [11]:
!ln -s /kaggle/input/ecapa-weight /kaggle/working/SASV_DoubleRoc-main/weight_ecapa

In [12]:
!ln -s /kaggle/input/cm-weight-new /kaggle/working/SASV_DoubleRoc-main/cm_weight_new

In [13]:
!ln -s /kaggle/input/pkl-s2pecnet-data /kaggle/working/SASV_DoubleRoc-main/pkl-s2pecnet-data

In [14]:
!ln -s /kaggle/input/public-tst-pkl /kaggle/working/SASV_DoubleRoc-main/pubilc_tst_pkl

In [15]:
!ln -s /kaggle/input/embd-tst-new-ecapa /kaggle/working/SASV_DoubleRoc-main/embd_tst_new_ecapa

In [16]:
!ln -s /kaggle/input/embd-asnorm /kaggle/working/SASV_DoubleRoc-main/embd_asnorm

In [17]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py
import os
import tqdm
import torch
import argparse
import numpy as np
import pickle as pk
import torch.nn as nn
from typing import Dict, List
import torch.nn.functional as F
from torch.utils.data import DataLoader
from test_dataset import get_dataloader
from models.ResNet34V2 import MainModel as ResNet34V2
from models.ECAPATDNN import MainModel as ECAPATDNN
from models.Res2Net50V2 import MainModel as Res2Net50V2
from models.Res2NeXt50 import MainModel as Res2NeXt50
from models.WavLMLarge import MainModel as WavLMLarge

vn_celeb_trn_path = "/kaggle/working/SASV_DoubleRoc-main/vn_celeb_txt/train_list_final.txt"
vn_celeb_tst_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
cm_tst_bf = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
 
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--LA_dataset_path', type=str, default='LA', help='The model to Run')
    parser.add_argument(
        "--ASV_embedings_save_path", type=str, default="embeddings"
    )
    parser.add_argument(
        "--model", type=str, default="ECAPATDNN"
    )
    parser.add_argument(
        "--gpu", type=str, default="0"
    )
    return parser.parse_args()


if __name__ == '__main__':
    args = get_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device: {}".format(device))
    LA_dataset_path = args.LA_dataset_path
    if not os.path.exists(LA_dataset_path):
        print('LA Datset is not exist.')
        exit(0)
    if args.model == 'ECAPATDNN':
        model = ECAPATDNN('/kaggle/working/SASV_DoubleRoc-main/weight_ecapa/model_0100.model')
    elif args.model == 'ResNet34V2':
        model = ResNet34V2('./weights/resnet34v2.model')
    elif args.model == 'Res2Net50V2':
        model = Res2Net50V2('./weights/res2net50v2.model')
    elif args.model == 'Res2NeXt50':
        model = Res2NeXt50('./weights/res2next50.model')
    elif args.model == 'WavLMLarge':
        model = WavLMLarge('./weights/wavlm_large_finetune.model')
    else:
        print(f'model {args.model} is not supoort.')
        exit(0)
    model = model.to(device)
    model = model.eval()
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    #for data_dirname in ['train_celeb', 'valid_celeb', 'enr_set']:
    #if data_dirname == "train_celeb":
    #    meta_lines = open(vn_celeb_trn_path, "r").readlines()
    #if data_dirname == "valid_celeb":
    meta_lines = open(vn_celeb_tst_path, "r").readlines()
    #elif data_dirname == "enr_set":
    utt_enr_list = []
    utt_tst_list = []
    for line in meta_lines:
        tmp = line.strip().split('\t')
        utt = tmp[0]
        spk = tmp[1]
        if utt not in utt_tst_list:
            utt_tst_list.append(utt)
        if spk not in utt_enr_list:
            utt_enr_list.append(spk)
    path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/"
    sasv_tst = get_dataloader(path, utt_tst_list)
    sasv_enr = get_dataloader(path, utt_enr_list)
    #for sasv_dataloader in ["sasv_tst", "sasv_enr"]:
    spk_emb_dic = {}
    with torch.no_grad():
        for idx, (data_1s, data_2s, files) in  tqdm.tqdm(enumerate(sasv_tst), total = len(sasv_tst)):
            data_1 = data_1s[0].cuda()
            data_2 = data_2s[0].cuda()
            file = files[0]
            embedding_1 = model(data_1)
            embedding_1_norm = F.normalize(embedding_1, p=2, dim=1)
            embedding_2 = model(data_2)
            embedding_2_norm  = F.normalize(embedding_2, p=2, dim=1)
            embedding = torch.cat([embedding_1, embedding_2], dim=0).detach().cpu().numpy()
            embedding_norm = torch.cat([embedding_1_norm, embedding_2_norm], dim=0).detach().cpu().numpy()
            spk_emb_dic[file] = embedding_norm
            #spk_emb_norm_dic[file] = embedding_norm
    os.makedirs(os.path.join(args.ASV_embedings_save_path, args.model), exist_ok=True)
    with open( f"{os.path.join(args.ASV_embedings_save_path, args.model)}/sasv_tst.pk", "wb") as f:
        pk.dump(spk_emb_dic, f)
        
    spk_emb_dic = {}
    with torch.no_grad():
        for idx, (data_1s, data_2s, files) in  tqdm.tqdm(enumerate(sasv_enr), total = len(sasv_enr)):
            data_1 = data_1s[0].cuda()
            data_2 = data_2s[0].cuda()
            file = files[0]

            embedding_1 = model(data_1)
            embedding_1_norm = F.normalize(embedding_1, p=2, dim=1)
            embedding_2 = model(data_2)
            embedding_2_norm  = F.normalize(embedding_2, p=2, dim=1)
            embedding = torch.cat([embedding_1, embedding_2], dim=0).detach().cpu().numpy()
            embedding_norm = torch.cat([embedding_1_norm, embedding_2_norm], dim=0).detach().cpu().numpy()
            spk_emb_dic[file] = embedding_norm
            #spk_emb_norm_dic[file] = embedding_norm   
    with open( f"{os.path.join(args.ASV_embedings_save_path, args.model)}/sasv_enr.pk", "wb") as f:
        pk.dump(spk_emb_dic, f)



Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py


In [18]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/test_dataset.py
import os
import torch
import numpy as np
from glob import glob
import soundfile as sf
from torch.utils.data import Dataset, DataLoader

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def transform_string(dong):
    # Sử dụng biểu thức chính quy để tìm các phần tử id trong dòng
    match = re.match(r'BF_id(\d{5})_(\d{8})', dong)
    
    if match:
        # Lấy các phần tử id từ kết quả tìm được
        id1, id2 = match.groups()
        # Chuyển đổi sang định dạng mong muốn
        ket_qua = f'id{id1.zfill(5)}/{id2.zfill(8)}'
        return ket_qua
    else:
        # Trả về dòng ban đầu nếu không tìm thấy kết quả phù hợp
        return dong


class SASVTestDataset(Dataset):
    def __init__(self, eval_path, list_id, num_frames=300):
        super().__init__()
        self.eval_path = eval_path
        self.list_id = list_id
        self.num_frames = num_frames
    
    def __len__(self):
        return len(self.list_id)
    
    def __getitem__(self, index):
        # return super().__getitem__(index)
        file = self.list_id[index]
        audio_path = self.eval_path + f"{file}"
        audio, _  = sf.read(audio_path)
        # Full utterance 完整的一句
        data_1 = torch.FloatTensor(np.stack([audio],axis=0))

        # Spliited utterance matrix 将语音切片
        max_audio = self.num_frames * 160 + 240
        if audio.shape[0] <= max_audio:
            shortage = max_audio - audio.shape[0]
            audio = np.pad(audio, (0, shortage), 'wrap')
        feats = []
        startframe = np.linspace(0, audio.shape[0]-max_audio, num=5)
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])
        feats = np.stack(feats, axis = 0).astype(np.float)
        data_2 = torch.FloatTensor(feats)
        return data_1, data_2, file

def get_dataloader(eval_path, list_id):
    sasv_dataset = SASVTestDataset(eval_path, list_id)
    sasv_dataloader = DataLoader(dataset=sasv_dataset, batch_size=1, num_workers=4)
    return sasv_dataloader





Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/test_dataset.py


In [19]:
!python /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py --model ECAPATDNN --LA_dataset_path "/kaggle/working/SASV_DoubleRoc-main/public_tst/"

Device: cuda
spk model load model_0100.model
100%|███████████████████████████████████████| 4766/4766 [01:46<00:00, 44.60it/s]


In [20]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py
import os
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
import soundfile as sf
#from scipy.io import wavfile
#import scipy
#import scipy.fft
#import librosa
from torch import Tensor
import torch, time, sys, tqdm
from utils import get_all_EERs
from models.AASIST import MainModel as CMModel
from models.S2pecNet import MainModel as S2pecNet
from torch.utils.data import Dataset, DataLoader

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--LA_dataset_path', type=str, default='/kaggle/working/SASV_DoubleRoc-main/public_tst/', help='The model to Run')
    parser.add_argument(
        "--ASV_embedings_save_path", type=str, default="/kaggle/working/embeddings/ECAPATDNN/"
    )
    parser.add_argument(
        "--model", type=str, default="ECAPATDNN"
    )
    parser.add_argument(
        "--norm", type=str, default="l2norm"
    )
    parser.add_argument(
        "--gpu", type=str, default="0"
    )
    return parser.parse_args()

def init_cm_model(cm_weight_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    cm_model = CMModel(cm_weight_path)
    cm_model = cm_model.to(device)
    cm_model = cm_model.eval()
    for module in cm_model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    return cm_model

def init_cm_model_1(cm_weight_path):
    device = "cuda" #if torch.cuda.is_available() else "cpu"
    cm_model = S2pecNet(cm_weight_path)
    cm_model = cm_model.to(device)
    cm_model = cm_model.eval()
    for module in cm_model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    return cm_model

def transform_string(input_string):
    # Tách lấy phần cuối của đường dẫn (tức là tên tệp tin)
    file_name = os.path.basename(input_string)
    
    # Loại bỏ phần mở rộng '.wav' và thay thế bằng 'LFCC.pkl'
    transformed_name = file_name.replace('.wav', 'LFCC.pkl')
    
    return transformed_name

'''
def stft(data,fs,window_length_ms=30,window_step_ms=20,windowing_function=None):
    window_length = int(window_length_ms*fs/2000)*2
    window_step = int(window_step_ms*fs/1000)
    if windowing_function is None:
        windowing_function = np.sin(np.pi*np.arange(0.5,window_length,1)/window_length)**2
    
    total_length = len(data)
    window_count = int( (total_length-window_length)/window_step) + 1
    
    spectrum_length = int((window_length)/2)+1
    spectrogram = np.zeros((window_count,spectrum_length),dtype=np.complex128)

    for k in range(window_count):
        starting_position = k*window_step

        data_vector = data[starting_position:(starting_position+window_length),]
        if len(windowing_function) != len(data_vector):
            windowing_function = np.sin(np.pi * np.arange(0.5, len(data_vector) + 0.5) / len(data_vector))
        window_spectrum = scipy.fft.rfft(data_vector*windowing_function,n=window_length)

        spectrogram[k,:] = window_spectrum
        
    return spectrogram

def istft(spectrogram,fs,window_length_ms=30,window_step_ms=20,windowing_function=None):
    window_length = int(window_length_ms*fs/2000)*2
    window_step = int(window_step_ms*fs/1000)
    #if windowing_function is None:
    #    windowing_function = np.ones(window_length)
    window_count = spectrogram.shape[0]
    
    total_length = (window_count-1)*window_step + window_length
    data = np.zeros(total_length)
    
    for k in range(window_count):
        starting_position = k*window_step
        ix = np.arange(starting_position,starting_position+window_length)

        thiswin = scipy.fft.irfft(spectrogram[k,:],n=window_length)
        if len(windowing_function) != len(thiswin):
            windowing_function = np.sin(np.pi * np.arange(0.5, len(thiswin) + 0.5) / len(thiswin))
    
        data[ix] = data[ix] + thiswin*windowing_function
        
    return data


def halfsinewindow(window_length):
    return np.sin(np.pi*np.arange(0.5,window_length,1)/window_length)

def zcr(data,fs,window_length_ms=30,window_step_ms=20):
    window_length = int(window_length_ms*fs/1000)
    window_step = int(window_step_ms*fs/1000)
    windowing_function = np.sin(np.pi*np.arange(0.5,window_length,1)/window_length)**2
    
    total_length = len(data)
    window_count = int( (total_length-window_length)/window_step) + 1

    y = np.zeros(window_count)

    for k in range(window_count):
        starting_position = k*window_step

        data_vector = data[starting_position:(starting_position+window_length),]
        y[k] = np.sum(np.abs(np.diff(np.sign(data_vector))))
        
    return y

def reduce_noise(filepath):
    fs = 64600  # Sample rate
    seconds = 5  # Duration of recording
    window_length_ms=30
    window_step_ms=15
    window_length = int(window_length_ms*fs/2000)*2
    window_step_samples = int(window_step_ms*fs/1000)

    windowing_function = halfsinewindow(window_length)
    fs, data = wavfile.read(filepath)
    data = data[:]
    spectrogram_matrix = stft(data,
                          fs,
                          window_length_ms=window_length_ms,
                          window_step_ms=window_step_ms,
                         windowing_function=windowing_function)
    fft_length = spectrogram_matrix.shape[1]
    window_count = spectrogram_matrix.shape[0]
    length_in_s = window_count*window_step_ms/1000
    
    # VAD through energy thresholding
    frame_energy = np.sum(np.abs(spectrogram_matrix)**2,axis=1)
    frame_energy_dB = 10*np.log10(frame_energy)
    mean_energy_dB = np.mean(frame_energy_dB) # mean of energy in dB

    noise_threshold_dB = mean_energy_dB - 3. # threshold relative to mean

    noise_active = frame_energy_dB < noise_threshold_dB
    noise_frames = spectrogram_matrix[noise_active,:]
    noise_estimate = np.mean(np.abs(noise_frames)**2,axis=0)
    energy_enhanced = np.subtract(np.abs(spectrogram_matrix)**2, np.expand_dims(noise_estimate,axis=0))
    energy_enhanced *= (energy_enhanced > 0)  # threshold at zero
    mmse_gain = energy_enhanced/(np.abs(spectrogram_matrix)**2)
    spectrogram_enhanced_mmse = spectrogram_matrix*mmse_gain
    
    data_enhanced_mmse = istft(spectrogram_enhanced_mmse,fs,window_length_ms=window_length_ms,window_step_ms=window_step_ms,windowing_function=windowing_function)
    return data_enhanced_mmse
'''

def asnorm_score(embd_trn, train_cohort):
    # Tính điểm số cho các cặp thử nghiệm và lưu vào một tensor
    cs = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
    mean_e_list = []
    mean_t_list = []
    std_e_list = []
    std_t_list = []
    for i in range(0, len(selected_trials), 2):
        try:
            enrol = torch.FloatTensor(embd_trn[selected_trials[i+1]])
            test = torch.FloatTensor(embd_trn[selected_trials[i]])
        except:
            enrol = (embd_trn[selected_trials[i+1]])
            test = (embd_trn[selected_trials[i]])

        # Getting norm stats for enrol impostors
        enrol_rep = enrol.repeat(train_cohort.shape[0], 1, 1)
        score_e_c = cs(enrol_rep.to(device), train_cohort.to(device)).detach().cpu().numpy()

        if "cohort_size" in params:
            score_e_c = torch.topk(
                torch.Tensor(score_e_c), k=params["cohort_size"], dim=0
            )[0].detach().cpu().numpy()

        mean_e_c = torch.mean(torch.FloatTensor(score_e_c), dim=0)
        std_e_c = torch.std(torch.FloatTensor(score_e_c), dim=0)
        mean_e_list.append(mean_e_c)
        std_e_list.append(std_e_c)
        # Getting norm stats for test impostors
        test_rep = test.repeat(train_cohort.shape[0], 1, 1)
        score_t_c = cs(test_rep.to(device), train_cohort.to(device)).detach().cpu().numpy()

        if "cohort_size" in params:
            score_t_c = torch.topk(torch.Tensor(score_t_c), k=params["cohort_size"], dim=0)[0].detach().cpu().numpy()

        mean_t_c = torch.mean(torch.FloatTensor(score_t_c), dim=0)
        std_t_c = torch.std(torch.FloatTensor(score_t_c), dim=0)
        mean_t_list.append(mean_t_c)
        std_t_list.append(std_t_c)

    return mean_e_list, mean_t_list, std_e_list, std_t_list

def cal_norm_score(mean_e_list, mean_t_list, std_e_list, std_t_list, asv_score):
    norm_score = []
    scores = []
    asv_score_float = torch.tensor(asv_score, dtype=torch.float)
    for (me, mt, se, st) in zip(mean_e_list, mean_t_list, std_e_list, std_t_list):
        score_e = (asv_score_float - me) / se
        score_t = (asv_score_float - mt) / st
        score_e = torch.mean(score_e)
        score_t = torch.mean(score_t)
        
        score = 0.5 * (score_e + score_t)
        norm_score.append(score.item())
    scores.append(np.array(norm_score).mean())    
    return scores, norm_score

def repeat_padding(spec, ref_len):
    mul = int(np.ceil(ref_len / spec.shape[1]))
    spec = spec.repeat(1, mul)[:, :ref_len]
    return spec

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

class Dataset_devNeval(Dataset):
    def __init__(self, base_dir, items, enr_emb, asv_embd, pkl_list, pkl_dir, trick_list):
        self.base_dir = base_dir
        self.items = items
        self.cut = 64600  # take ~4 sec audio (64600 samples), 这个对于CM系统可能需要修改
        self.asv_embd = asv_embd
        self.enr_emb = enr_emb
        self.pkl_list = pkl_list
        self.pkl_dir = pkl_dir
        self.trick_list = trick_list
        for key in asv_embd:
            asv_embd[key] = asv_embd[key].mean(0)
        for key in enr_emb:
            enr_emb[key] = enr_emb[key].mean(0)
        
    def __len__(self):
        return len(self.items)

    def __getitem__(self, index):
        spk, tst = self.items[index]
        if tst in self.trick_list:
            flag = 0
        else:
            flag = 1
        X, _ = sf.read('{}{}'.format(self.base_dir, tst))
        Y, _ = sf.read('{}{}'.format(self.base_dir, spk))
        #X = reduce_noise('{}{}'.format(self.base_dir, tst))
        #Y = reduce_noise('{}{}'.format(self.base_dir, spk))
        pkl_tst_id = transform_string(tst)
        pkl_enr_id = transform_string(spk)
        with open(self.pkl_list[pkl_tst_id], 'rb') as f:
            lfcc_tst = pickle.load(f)
        with open(self.pkl_list[pkl_enr_id], 'rb') as f:
            lfcc_enr = pickle.load(f)            
        lfcc_tst = torch.from_numpy(lfcc_tst)
        lfcc_enr = torch.from_numpy(lfcc_enr)
        this_tst_len = lfcc_tst.shape[1]
        this_enr_len = lfcc_enr.shape[1]
        if this_tst_len > 750:
            lfcc_tst = lfcc_tst[:, :750]
        if this_enr_len > 750:
            lfcc_enr = lfcc_enr[:, :750]
        if this_tst_len < 750:
            lfcc_tst = repeat_padding(lfcc_tst, 750)
        if this_enr_len < 750:
            lfcc_enr = repeat_padding(lfcc_enr, 750)            
        lfcc_tst = lfcc_tst.float()
        lfcc_enr = lfcc_enr.float()
        X_pad = pad(X, self.cut)
        x_inp = Tensor(X_pad)
        Y_pad = pad(Y, self.cut)
        y_inp = Tensor(Y_pad)
        return self.enr_emb[spk], self.asv_embd[tst], x_inp, y_inp, lfcc_tst, lfcc_enr, index, flag

def gen_dev_and_eval_item_list(meta_path):
    items = []
    with open(meta_path, 'r', encoding='utf-8') as fin:
        for line in fin:
            tmp = line.strip().split('\t')
            tst = tmp[0]
            enr = tmp[1]
            items.append([enr, tst])
    return items


def gen_loader(database_path, embedding_path, batch_size, n_cpu, norm):

    dev_database_path  = database_path + "/ASVspoof2019_LA_dev/"
    eval_database_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/"

    dev_trial_path  = database_path + "/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.gi.trl.txt"
    eval_trial_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
    eval_pk_path = "/kaggle/working/SASV_DoubleRoc-main/pubilc_tst_pkl/PKL_public_test/eval/"
    # dev dataset setting
    # dev_spk2ids, dev_items = gen_dev_and_eval_item_list([database_path + '/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.male.trn.txt',
    #                                                      database_path + '/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.female.trn.txt'],
    #                                                     dev_trial_path)
    # dev_asv_embd = pickle.load(open(os.path.join(embedding_path, f'ASVspoof2019_LA_dev_{norm}.pk'), 'rb'))

    # dev_dataset = Dataset_devNeval(base_dir=dev_database_path, spk2ids=dev_spk2ids, items=dev_items, asv_embd=dev_asv_embd)
    # dev_loader = DataLoader(dataset=dev_dataset,
    #                         batch_size=batch_size,
    #                         shuffle=False,
    #                         drop_last=False,
    #                         pin_memory=True,
    #                         num_workers=n_cpu)
    # eval dataset setting
    eval_items = gen_dev_and_eval_item_list(eval_trial_path)
    eval_asv_embd = pickle.load(open("/kaggle/working/SASV_DoubleRoc-main/embd_tst_new_ecapa/test_norm_res2net59.pk",'rb'))
    enr_emb = pickle.load(open("/kaggle/working/SASV_DoubleRoc-main/embd_tst_new_ecapa/enr_norm_res2net59.pk",'rb'))
    
    public_tst_pkl_list = {}
    files = os.listdir(eval_pk_path)
    for file in files:
        if os.path.isfile(os.path.join(eval_pk_path, file)):
            public_tst_pkl_list[file] = os.path.join(eval_pk_path, file)

    meta_lines = open(eval_trial_path, "r").readlines()
    #elif data_dirname == "enr_set":
    utt_enr_list = []
    utt_tst_list = []
    trick_list = []
    for line in meta_lines:
        tmp = line.strip().split('\t')
        utt = tmp[0]
        spk = tmp[1]
        if utt not in utt_tst_list:
            utt_tst_list.append(utt)
        if spk not in utt_enr_list:
            utt_enr_list.append(spk)
    for spk in utt_enr_list:
        if spk in utt_tst_list:
            trick_list.append(spk)
    flag = {}
    eval_dataset = Dataset_devNeval(base_dir=eval_database_path, items=eval_items, enr_emb = enr_emb, asv_embd=eval_asv_embd, pkl_list = public_tst_pkl_list, pkl_dir = eval_pk_path, trick_list = trick_list)
    eval_loader = DataLoader(dataset=eval_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False,
                             pin_memory=True,
                             num_workers=n_cpu)
    return eval_loader

if __name__ == '__main__':
    args = get_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    device = "cuda" #if torch.cuda.is_available() else "cpu"
    print("Device: {}".format(device))
    print(f'Model: {args.model}')
    print(f'Norm: {args.norm}')
    assert args.norm in ['l2norm', 'nonorm']
    LA_dataset_path = args.LA_dataset_path
    ASV_embedings_save_path = args.ASV_embedings_save_path
    cm_model = init_cm_model_1('/kaggle/working/SASV_DoubleRoc-main/cm_weight_new/epoch_45_0.041.pth')
    cos = nn.CosineSimilarity(dim=1, eps=1e-6).to(device)
    eval_loader = gen_loader(LA_dataset_path, os.path.join(ASV_embedings_save_path, args.model), batch_size=64, n_cpu=4, norm=args.norm)
    tag_2_loader = {'eval': eval_loader}
    params={
        "cohort_size": 1200
    }
    with open("/kaggle/working/SASV_DoubleRoc-main/embd_asnorm/train_1200.pk", "rb") as f:
        embd_trn = pickle.load(f)
    N = 300
    selected_trials = list(embd_trn.keys())[:N * 2]
    tensor_list = [torch.FloatTensor(arr) for arr in embd_trn.values()]
    # Sử dụng torch.stack để tạo ra một tensor từ list của các FloatTensors
    train_cohort = torch.stack(tensor_list)
    for tag in tag_2_loader.keys():
        loader = tag_2_loader[tag]
        id2score = {}
        id2label = {}
        id2score_no_trick = {}
        id2score_no_trick1 = {}
        scores_list, labels_list = [], []
        asv_score_list = []
        score_no_trick_list = []
        bonafide_no_trick = []
        bonafide_trick = []
        for idx, (asv_embd_enr, asv_embd_tst, fusion_cm_data, enr_audio, lfcc_tst, lfcc_enr, index, flag) in enumerate(loader):
            with torch.no_grad():
                index = index.detach().cpu().numpy()
                #flag = flag.detach().cpu().numpy()
                print(flag)
                tst_dec = cm_model.forward(fusion_cm_data.to(device), lfcc_tst.to(device))[0]
                enr_dec = cm_model.forward(enr_audio.to(device), lfcc_enr.to(device))[0]
                cm_cos = cos(tst_dec.to(device), enr_dec.to(device)).detach().cpu().numpy()
                asv_cos = cos(asv_embd_enr.to(device), asv_embd_tst.to(device)).detach().cpu().numpy()
                #asv_cos = cos(asv_embd_enr.to(device), asv_embd_tst.to(device)).detach().cpu().numpy()    
                #score = cm_cos * asv_cos
    
                
                #asv_cos_1 = (asv_cos + 1)/2
                output = cm_model.forward(fusion_cm_data.to(device), lfcc_tst.to(device))[1]
                sigmoid_output = torch.sigmoid(output).detach().cpu().numpy()[:, 1]
                #score1 = output.softmax(-1).detach().cpu().numpy()[:, 1]
                #score2 = asv_cos_1 * score1
                mean_e_list, mean_t_list, std_e_list, std_t_list = asnorm_score(embd_trn, train_cohort)
                for i, s, l, m, f in zip(index, asv_cos, cm_cos, sigmoid_output, flag):
                    asv_norm_score, norm_score = cal_norm_score(mean_e_list, mean_t_list, std_e_list, std_t_list, s)
                    asv_scaled = (asv_norm_score - np.min(norm_score)) / (np.max(norm_score) - np.min(norm_score))
                    asv_scaled = asv_scaled.item()                   
                    p = 1 - m
                    bonafide_no_trick.append(m)
                    score_no_trick = asv_scaled * s * l * p
                    final_no_trick = max(0, score_no_trick)
                    id2score_no_trick[i] = final_no_trick
                    asv_score_list.append(asv_scaled)
                    if f == 0:
                        score = (asv_scaled + s)/2
                        score1 = asv_scaled * s
                        final = max(0, score)
                        final1 = max(0, score1)
                        id2score[i] = final
                        id2label[i] = final1
                        bonafide_trick.append(1)
                    else:
                        score = (asv_scaled + s + l + p)/4
                        score1 = asv_scaled * s * l * p
                        final = max(0, score)
                        final1 = max(0, score1)
                        id2score[i] = final
                        id2label[i] = final1
                        bonafide_trick.append(m)

                '''
                #index_tuple = tuple(index)
                for i, s, l, f in zip(index, score, asv_cos, flag):
                    if f == 0:
                        final_score = max(0, l)
                        id2score[i] = final_score 
                    else:
                        final_score = max(0, s)
                        id2score[i] = final_score 
                        
                for i, s, l, f in zip(index, score2, asv_cos_1, flag):
                    if f == 0:
                        id2label[i] = l
                    else:
                        id2label[i] = s
                    asv_score_list.append(l)
                '''
        for i in range(len(id2score.keys())):
            scores_list.append(id2score[i])
            labels_list.append(id2label[i])
            score_no_trick_list.append(id2score_no_trick[i])

with open("/kaggle/working/SASV_DoubleRoc-main/result.txt", 'w') as f:
    for item in scores_list:
        f.write("%s\n" %item)
with open("/kaggle/working/SASV_DoubleRoc-main/result_noCM.txt", 'w') as f:
    for item in labels_list:
        f.write("%s\n" %item)
with open("/kaggle/working/SASV_DoubleRoc-main/score_no_trick.txt", 'w') as f:
    for item in score_no_trick_list:
        f.write("%s\n" %item)  
with open("/kaggle/working/SASV_DoubleRoc-main/bonafide_no_trick.txt", 'w') as f:
    for item in bonafide_no_trick:
        f.write("%s\n" %item)  
with open("/kaggle/working/SASV_DoubleRoc-main/bonafide_trick.txt", 'w') as f:
    for item in bonafide_trick:
        f.write("%s\n" %item)  
with open("/kaggle/working/SASV_DoubleRoc-main/asv_score.txt", 'w') as f:
    for item in asv_score_list:
        f.write("%s\n" %item)  

Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py


In [21]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/models/S2pecNet.py
import random
from typing import Union
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from models.resnet import ResNet
from models.decoder import *

index = None

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_dim, out_dim, **kwargs):
        super().__init__()

        # attention map
        self.att_proj = nn.Linear(in_dim, out_dim)
        self.att_weight = self._init_new_params(out_dim, 1)

        # project
        self.proj_with_att = nn.Linear(in_dim, out_dim)
        self.proj_without_att = nn.Linear(in_dim, out_dim)

        # batch norm
        self.bn = nn.BatchNorm1d(out_dim)

        # dropout for inputs
        self.input_drop = nn.Dropout(p=0.2)

        # activate
        self.act = nn.SELU(inplace=True)

        # temperature
        self.temp = 1.
        if "temperature" in kwargs:
            self.temp = kwargs["temperature"]

    def forward(self, x):
        '''
        x   :(#bs, #node, #dim)
        '''
        # apply input dropout
        x = self.input_drop(x)

        # derive attention map
        att_map = self._derive_att_map(x)

        # projection
        x = self._project(x, att_map)

        # apply batch norm
        x = self._apply_BN(x)
        x = self.act(x)
        return x

    def _pairwise_mul_nodes(self, x):
        '''
        Calculates pairwise multiplication of nodes.
        - for attention map
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, #dim)
        '''

        nb_nodes = x.size(1)
        x = x.unsqueeze(2).expand(-1, -1, nb_nodes, -1)
        x_mirror = x.transpose(1, 2)

        return x * x_mirror

    def _derive_att_map(self, x):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = self._pairwise_mul_nodes(x)
        # size: (#bs, #node, #node, #dim_out)
        att_map = torch.tanh(self.att_proj(att_map))
        # size: (#bs, #node, #node, 1)
        att_map = torch.matmul(att_map, self.att_weight)

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _project(self, x, att_map):
        x1 = self.proj_with_att(torch.matmul(att_map.squeeze(-1), x))
        x2 = self.proj_without_att(x)

        return x1 + x2

    def _apply_BN(self, x):
        org_size = x.size()
        x = x.view(-1, org_size[-1])
        x = self.bn(x)
        x = x.view(org_size)

        return x

    def _init_new_params(self, *size):
        out = nn.Parameter(torch.FloatTensor(*size))
        nn.init.xavier_normal_(out)
        return out


class HtrgGraphAttentionLayer(nn.Module):
    def __init__(self, in_dim, out_dim, **kwargs):
        super().__init__()

        self.proj_type1 = nn.Linear(in_dim, in_dim)
        self.proj_type2 = nn.Linear(in_dim, in_dim)

        # attention map
        self.att_proj = nn.Linear(in_dim, out_dim)
        self.att_projM = nn.Linear(in_dim, out_dim)

        self.att_weight11 = self._init_new_params(out_dim, 1)
        self.att_weight22 = self._init_new_params(out_dim, 1)
        self.att_weight12 = self._init_new_params(out_dim, 1)
        self.att_weightM = self._init_new_params(out_dim, 1)

        # project
        self.proj_with_att = nn.Linear(in_dim, out_dim)
        self.proj_without_att = nn.Linear(in_dim, out_dim)

        self.proj_with_attM = nn.Linear(in_dim, out_dim)
        self.proj_without_attM = nn.Linear(in_dim, out_dim)

        # batch norm
        self.bn = nn.BatchNorm1d(out_dim)

        # dropout for inputs
        self.input_drop = nn.Dropout(p=0.2)

        # activate
        self.act = nn.SELU(inplace=True)

        # temperature
        self.temp = 1.
        if "temperature" in kwargs:
            self.temp = kwargs["temperature"]

    def forward(self, x1, x2, master=None):
        '''
        x1  :(#bs, #node, #dim)
        x2  :(#bs, #node, #dim)
        '''
        num_type1 = x1.size(1)
        num_type2 = x2.size(1)

        x1 = self.proj_type1(x1)
        x2 = self.proj_type2(x2)

        x = torch.cat([x1, x2], dim=1)

        if master is None:
            master = torch.mean(x, dim=1, keepdim=True)

        # apply input dropout
        x = self.input_drop(x)

        # derive attention map
        att_map = self._derive_att_map(x, num_type1, num_type2)

        # directional edge for master node
        master = self._update_master(x, master)

        # projection
        x = self._project(x, att_map)

        # apply batch norm
        x = self._apply_BN(x)
        x = self.act(x)

        x1 = x.narrow(1, 0, num_type1)
        x2 = x.narrow(1, num_type1, num_type2)

        return x1, x2, master

    def _update_master(self, x, master):

        att_map = self._derive_att_map_master(x, master)
        master = self._project_master(x, master, att_map)

        return master

    def _pairwise_mul_nodes(self, x):
        '''
        Calculates pairwise multiplication of nodes.
        - for attention map
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, #dim)
        '''

        nb_nodes = x.size(1)
        x = x.unsqueeze(2).expand(-1, -1, nb_nodes, -1)
        x_mirror = x.transpose(1, 2)

        return x * x_mirror

    def _derive_att_map_master(self, x, master):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = x * master
        att_map = torch.tanh(self.att_projM(att_map))

        att_map = torch.matmul(att_map, self.att_weightM)

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _derive_att_map(self, x, num_type1, num_type2):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = self._pairwise_mul_nodes(x)
        # size: (#bs, #node, #node, #dim_out)
        att_map = torch.tanh(self.att_proj(att_map))
        # size: (#bs, #node, #node, 1)

        att_board = torch.zeros_like(att_map[:, :, :, 0]).unsqueeze(-1)

        att_board[:, :num_type1, :num_type1, :] = torch.matmul(
            att_map[:, :num_type1, :num_type1, :], self.att_weight11)
        att_board[:, num_type1:, num_type1:, :] = torch.matmul(
            att_map[:, num_type1:, num_type1:, :], self.att_weight22)
        att_board[:, :num_type1, num_type1:, :] = torch.matmul(
            att_map[:, :num_type1, num_type1:, :], self.att_weight12)
        att_board[:, num_type1:, :num_type1, :] = torch.matmul(
            att_map[:, num_type1:, :num_type1, :], self.att_weight12)

        att_map = att_board

        # att_map = torch.matmul(att_map, self.att_weight12)

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _project(self, x, att_map):
        x1 = self.proj_with_att(torch.matmul(att_map.squeeze(-1), x))
        x2 = self.proj_without_att(x)

        return x1 + x2

    def _project_master(self, x, master, att_map):

        x1 = self.proj_with_attM(torch.matmul(
            att_map.squeeze(-1).unsqueeze(1), x))
        x2 = self.proj_without_attM(master)

        return x1 + x2

    def _apply_BN(self, x):
        org_size = x.size()
        x = x.view(-1, org_size[-1])
        x = self.bn(x)
        x = x.view(org_size)

        return x

    def _init_new_params(self, *size):
        out = nn.Parameter(torch.FloatTensor(*size))
        nn.init.xavier_normal_(out)
        return out


class GraphPool(nn.Module):
    def __init__(self, k: float, in_dim: int, p: Union[float, int]):
        super().__init__()
        self.k = k
        self.sigmoid = nn.Sigmoid()
        self.proj = nn.Linear(in_dim, 1)
        self.drop = nn.Dropout(p=p) if p > 0 else nn.Identity()
        self.in_dim = in_dim

    def forward(self, h):
        Z = self.drop(h)
        weights = self.proj(Z)
        scores = self.sigmoid(weights)
        new_h = self.top_k_graph(scores, h, self.k)

        return new_h

    def top_k_graph(self, scores, h, k):
        """
        args
        =====
        scores: attention-based weights (#bs, #node, 1)
        h: graph data (#bs, #node, #dim)
        k: ratio of remaining nodes, (float)

        returns
        =====
        h: graph pool applied data (#bs, #node', #dim)
        """
        _, n_nodes, n_feat = h.size()
        n_nodes = max(int(n_nodes * k), 1)
        _, idx = torch.topk(scores, n_nodes, dim=1)
        idx = idx.expand(-1, -1, n_feat)

        h = h * scores
        h = torch.gather(h, 1, idx)

        return h


class CONV(nn.Module):
    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10**(mel / 2595) - 1)

    def __init__(self,
                 out_channels,
                 kernel_size,
                 sample_rate=16000,
                 in_channels=1,
                 stride=1,
                 padding=0,
                 dilation=1,
                 bias=False,
                 groups=1,
                 mask=False):
        super().__init__()
        if in_channels != 1:

            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (
                in_channels)
            raise ValueError(msg)
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.sample_rate = sample_rate

        # Forcing the filters to be odd (i.e, perfectly symmetrics)
        if kernel_size % 2 == 0:
            self.kernel_size = self.kernel_size + 1
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.mask = mask
        if bias:
            raise ValueError('SincConv does not support bias.')
        if groups > 1:
            raise ValueError('SincConv does not support groups.')

        NFFT = 512
        f = int(self.sample_rate / 2) * np.linspace(0, 1, int(NFFT / 2) + 1)
        fmel = self.to_mel(f)
        fmelmax = np.max(fmel)
        fmelmin = np.min(fmel)
        filbandwidthsmel = np.linspace(fmelmin, fmelmax, self.out_channels + 1)
        filbandwidthsf = self.to_hz(filbandwidthsmel)

        self.mel = filbandwidthsf
        self.hsupp = torch.arange(-(self.kernel_size - 1) / 2,
                                  (self.kernel_size - 1) / 2 + 1)
        self.band_pass = torch.zeros(self.out_channels, self.kernel_size)
        for i in range(len(self.mel) - 1):
            fmin = self.mel[i]
            fmax = self.mel[i + 1]
            hHigh = (2*fmax/self.sample_rate) * \
                np.sinc(2*fmax*self.hsupp/self.sample_rate)
            hLow = (2*fmin/self.sample_rate) * \
                np.sinc(2*fmin*self.hsupp/self.sample_rate)
            hideal = hHigh - hLow

            self.band_pass[i, :] = Tensor(np.hamming(
                self.kernel_size)) * Tensor(hideal)

    def forward(self, x, mask=False):
        band_pass_filter = self.band_pass.clone().to(x.device)
        if mask:
            A = np.random.uniform(0, 20)
            A = int(A)
            A0 = random.randint(0, band_pass_filter.shape[0] - A)
            band_pass_filter[A0:A0 + A, :] = 0
        else:
            band_pass_filter = band_pass_filter

        self.filters = (band_pass_filter).view(self.out_channels, 1,
                                               self.kernel_size)

        return F.conv1d(x,
                        self.filters,
                        stride=self.stride,
                        padding=self.padding,
                        dilation=self.dilation,
                        bias=None,
                        groups=1)



class Residual_block(nn.Module):
    def __init__(self, nb_filts, first=False):
        super().__init__()
        self.first = first

        if not self.first:
            self.bn1 = nn.BatchNorm2d(num_features=nb_filts[0])
        self.conv1 = nn.Conv2d(in_channels=nb_filts[0],
                               out_channels=nb_filts[1],
                               kernel_size=(2, 3),
                               padding=(1, 1),
                               stride=1)
        self.selu = nn.SELU(inplace=True)

        self.bn2 = nn.BatchNorm2d(num_features=nb_filts[1])
        self.conv2 = nn.Conv2d(in_channels=nb_filts[1],
                               out_channels=nb_filts[1],
                               kernel_size=(2, 3),
                               padding=(0, 1),
                               stride=1)

        if nb_filts[0] != nb_filts[1]:
            self.downsample = True
            self.conv_downsample = nn.Conv2d(in_channels=nb_filts[0],
                                             out_channels=nb_filts[1],
                                             padding=(0, 1),
                                             kernel_size=(1, 3),
                                             stride=1)

        else:
            self.downsample = False
        self.mp = nn.MaxPool2d((1, 3))  # self.mp = nn.MaxPool2d((1,4))


    def forward(self, x):
        identity = x
        if not self.first:
            out = self.bn1(x)
            out = self.selu(out)
        else:
            out = x
        out = self.conv1(x)

        # print('out',out.shape)
        out = self.bn2(out)
        out = self.selu(out)
        # print('out',out.shape)
        out = self.conv2(out)
        #print('conv2 out',out.shape)
        if self.downsample:
            identity = self.conv_downsample(identity)

        out += identity
        out = self.mp(out)
        return out

class TSF(nn.Module):

    def __init__(self, channels , *args, **kwargs):
        super(TSF, self).__init__()

        self.conv0 = nn.Conv2d(channels*2, channels , kernel_size=1, padding=0)
        self.norm0 = nn.BatchNorm2d(channels)

        self.pooling1 = torch.nn.AdaptiveAvgPool2d((None,1))
        self.fc1 = nn.Conv2d(channels, channels, kernel_size=1, padding=0)
        self.norm1 = nn.BatchNorm2d(channels)
        self.silu = nn.SiLU()
        self.fc2 = nn.Conv2d(channels, channels, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()
        
        
        self.pooling2 = torch.nn.AdaptiveAvgPool2d((1,None))
        self.fc3 = nn.Conv2d(channels, channels, kernel_size=1, padding=0)
        self.norm2 = nn.BatchNorm2d(channels)
        self.fc4 = nn.Conv2d(channels , channels, kernel_size=1, padding=0)
        
        
        


    def forward(self, x , lfcc):
        x = torch.cat((x,lfcc),1)
        x = self.silu(self.norm0(self.conv0(x)))
        identity = x
        
        x_f = self.pooling1(x)
        x_f = self.fc1(x_f)
        x_f = self.norm1(x_f)
        x_f = self.silu(x_f)
        x_f = self.fc2(x_f)
        x_f = self.sigmoid(x_f)
        
        x_t = self.pooling2(x)
        x_t = self.fc3(x_t)
        x_t = self.norm2(x_t)
        x_t = self.silu(x_t)
        x_t = self.fc4(x_t)
        x_t = self.sigmoid(x_t)
        
        attention = x_f*x_t
        out = attention*(identity) + (identity)
       
        
        return out
    
class Model(nn.Module):
    def __init__(self,):
        super().__init__()
        

        filts = [70, [1, 32], [32, 32], [32, 64], [64, 64]]
        gat_dims = [64, 32]
        pool_ratios = [0.5, 0.7, 0.5, 0.5]
        temperatures = [2.0, 2.0, 100.0, 100.0]

        self.conv_time = CONV(out_channels=filts[0],
                              kernel_size=128,
                              in_channels=1)
        self.first_bn = nn.BatchNorm2d(num_features=1)

        self.drop = nn.Dropout(0.5, inplace=True)
        self.drop_way = nn.Dropout(0.2, inplace=True)
        self.selu = nn.SELU(inplace=True)

        self.encoder = nn.Sequential(
            nn.Sequential(Residual_block(nb_filts=filts[1], first=True)),
            nn.Sequential(Residual_block(nb_filts=filts[2])),
            nn.Sequential(Residual_block(nb_filts=filts[3])),
            nn.Sequential(Residual_block(nb_filts=filts[4])),
            nn.Sequential(Residual_block(nb_filts=filts[4])),
            nn.Sequential(Residual_block(nb_filts=filts[4])))

        self.pos_S = nn.Parameter(torch.randn(1, 23, filts[-1][-1]))
        self.master1 = nn.Parameter(torch.randn(1, 1, gat_dims[0]))
        self.master2 = nn.Parameter(torch.randn(1, 1, gat_dims[0]))

        self.GAT_layer_S = GraphAttentionLayer(filts[-1][-1],
                                               gat_dims[0],
                                               temperature=temperatures[0])
        self.GAT_layer_T = GraphAttentionLayer(filts[-1][-1],
                                               gat_dims[0],
                                               temperature=temperatures[1])

        self.HtrgGAT_layer_ST11 = HtrgGraphAttentionLayer(
            gat_dims[0], gat_dims[1], temperature=temperatures[2])
        self.HtrgGAT_layer_ST12 = HtrgGraphAttentionLayer(
            gat_dims[1], gat_dims[1], temperature=temperatures[2])

        self.HtrgGAT_layer_ST21 = HtrgGraphAttentionLayer(
            gat_dims[0], gat_dims[1], temperature=temperatures[2])

        self.HtrgGAT_layer_ST22 = HtrgGraphAttentionLayer(
            gat_dims[1], gat_dims[1], temperature=temperatures[2])

        self.pool_S = GraphPool(pool_ratios[0], gat_dims[0], 0.3)
        self.pool_T = GraphPool(pool_ratios[1], gat_dims[0], 0.3)
        self.pool_hS1 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)
        self.pool_hT1 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)

        self.pool_hS2 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)
        self.pool_hT2 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)

        self.out_layer = nn.Linear(5 * gat_dims[1], 2)
        
        self.resnet = ResNet(3,256)
        self.two = TSF(64)
        
        self.decoder = Decoder()
        self.raw_decoder = RawDecoder()
#         exit()

    def forward(self, x, lfcc, Freq_aug=False):
        feat = self.resnet(lfcc.unsqueeze(1))

        x = x.unsqueeze(1)
        x = self.conv_time(x, mask=Freq_aug)
        x = x.unsqueeze(dim=1)
        x = F.max_pool2d(torch.abs(x), (3, 3))
        x = self.first_bn(x)
        x = self.selu(x)

        # get embeddings using encoder
        # (#bs, #filt, #spec, #seq)
        e = self.encoder(x)
        e = self.two(e,feat)
        
        dec_raw = self.raw_decoder(e)
        dec_lfcc = self.decoder(e)

        # spectral GAT (GAT-S)
        e_S, _ = torch.max(torch.abs(e), dim=3)  # max along time
        e_S = e_S.transpose(1, 2) + self.pos_S

        gat_S = self.GAT_layer_S(e_S)
        out_S = self.pool_S(gat_S)  # (#bs, #node, #dim)

        # temporal GAT (GAT-T)
        e_T, _ = torch.max(torch.abs(e), dim=2)  # max along freq
        e_T = e_T.transpose(1, 2)

        gat_T = self.GAT_layer_T(e_T)
        out_T = self.pool_T(gat_T)

        # learnable master node
        master1 = self.master1.expand(x.size(0), -1, -1)
        master2 = self.master2.expand(x.size(0), -1, -1)

        # inference 1
        out_T1, out_S1, master1 = self.HtrgGAT_layer_ST11(
            out_T, out_S, master=self.master1)

        out_S1 = self.pool_hS1(out_S1)
        out_T1 = self.pool_hT1(out_T1)

        out_T_aug, out_S_aug, master_aug = self.HtrgGAT_layer_ST12(
            out_T1, out_S1, master=master1)
        out_T1 = out_T1 + out_T_aug
        out_S1 = out_S1 + out_S_aug
        master1 = master1 + master_aug

        # inference 2
        out_T2, out_S2, master2 = self.HtrgGAT_layer_ST21(
            out_T, out_S, master=self.master2)
        out_S2 = self.pool_hS2(out_S2)
        out_T2 = self.pool_hT2(out_T2)

        out_T_aug, out_S_aug, master_aug = self.HtrgGAT_layer_ST22(
            out_T2, out_S2, master=master2)
        out_T2 = out_T2 + out_T_aug
        out_S2 = out_S2 + out_S_aug
        master2 = master2 + master_aug

        out_T1 = self.drop_way(out_T1)
        out_T2 = self.drop_way(out_T2)
        out_S1 = self.drop_way(out_S1)
        out_S2 = self.drop_way(out_S2)
        master1 = self.drop_way(master1)
        master2 = self.drop_way(master2)

        out_T = torch.max(out_T1, out_T2)
        out_S = torch.max(out_S1, out_S2)
        master = torch.max(master1, master2)

        T_max, _ = torch.max(torch.abs(out_T), dim=1)
        T_avg = torch.mean(out_T, dim=1)

        S_max, _ = torch.max(torch.abs(out_S), dim=1)
        S_avg = torch.mean(out_S, dim=1)

        last_hidden = torch.cat(
            [T_max, T_avg, S_max, S_avg, master.squeeze(1)], dim=1)

        last_hidden = self.drop(last_hidden)
        output = self.out_layer(last_hidden)

        return last_hidden, output, dec_raw, dec_lfcc
    

def resume_cm_model(model_file_path, network):
    model_dict = torch.load(model_file_path, map_location='cpu')
    network.load_state_dict(model_dict['model'])
    print(f'cm model load {os.path.split(model_file_path)[-1]}')

def MainModel(model_file_path):
    model = Model()
    resume_cm_model(model_file_path, model)
    return model

Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/models/S2pecNet.py


In [22]:
!python /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py --norm l2norm --model ECAPATDNN 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Device: cuda
Model: ECAPATDNN
Norm: l2norm
cm model load epoch_45_0.041.pth
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 